In [1]:
%pip install langchain
%pip install -U langchain-community
%pip install tiktoken
%pip install streamlit
%pip install unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [3]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [4]:
# 导入网站信息
urls = ["https://www.societyforscience.org/regeneron-sts/2025-scholars/"]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

In [7]:
data

[Document(metadata={'source': 'https://www.societyforscience.org/regeneron-sts/2025-scholars/'}, page_content='Congratulations to the 2025 Regeneron STS Scholars!\n\nSociety for Science proudly announces the top 300 scholars in the Regeneron Science Talent Search 2025, the nation’s oldest and most prestigious science and math competition for high school seniors. The Regeneron Science Talent Search provides students a national stage to present original research and celebrates the hard work and novel discoveries of young scientists who are bringing a fresh perspective to significant global challenges. The 300 scholars and their schools will be awarded $2,000 each.\n\nScholars were chosen based on their outstanding research, leadership skills, community involvement, commitment to academics, creativity in asking scientific questions and exceptional promise as STEM leaders demonstrated through the submission of their original, independent research projects, essays and recommendations. The 3

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n"], chunk_size=1500, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [9]:
len(texts)

41

In [10]:
texts[40].page_content

'Ashley Zhu, Age: 17 Hunter College High School, NY Project Title: On Lobe Disappearance of n-Loop Curves Under Curve Shortening Flow\n\nMinghao Zou, Age: 18 Valley Christian High School, CA Project Title: A New Particle Pusher With Hadronic Interactions for Modeling Multimessenger Emission From Compact Objects\n\nIvan Zou, Age: 17 Stuyvesant High School, NY Project Title: Targeting CDK7: Comparing the Effects of Inhibition, Degradation and Phosphorylation Mutants'

In [11]:
import os

os.environ["OPENAI_API_BASE"] = "your_api_base"
os.environ["OPENAI_API_KEY"] = "your_api_key"

In [12]:
from langchain.prompts import PromptTemplate

template = """Between >>> and <<< are the content.
The content is from science talent search website and it includes TOP 300 participants' information.
Your work is extract some information(Name, Age, School, State and Project Title) from the content. if you can not find relevant information, you do not output anything.
You should extract them in JSON format and do no miss anything from the content

>>> {page_content} <<<

Here is an example Use the following JSON format to include all the information. Pay attention to the State
[
  {{
    'Name': 'Maya Abdelaal',
    'Age': '18',
    'School': 'Marine Academy of Technology & Environmental Science',
    'State': 'NJ',
    'Project Title': 'Assessing Airborne Bacteria Abundance in Sea Foam Aerosols'
  }},
  {{
    'Name': 'Laasya Acharya',
    'Age': '17',
    'School': 'William Mason High School',
    'State': 'OH',
    'Project Title': 'Implementing a Novel Multimodal Neural Network Approach Using Dynamic Hyperparameter Selection Within an Unmanned Aerial Vehicle for the Early Detection of Crop Diseases'
    }},
]
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["page_content"],
    template=template,)

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model="gpt-4", temperature=0)

<ipython-input-13-6b39d6d98909>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


In [14]:
chain = PROMPT | llm | StrOutputParser()

In [15]:
# 这里有问题
# df = pd.DataFrame()
data_list = []

for t in texts[:3]:
  inputs = {
            "page_content": t.page_content,
            }
  res = chain.invoke(input=inputs)
  data_list.append(res)

In [51]:
import pandas as pd

df = pd.DataFrame()
for i,f in enumerate(data_list):
  new_df = pd.read_json(f.strip('`').lstrip('json').strip())
  df = pd.concat([new_df, df], ignore_index=True)

<ipython-input-51-a66ee8866337>:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  new_df = pd.read_json(f.strip('`').lstrip('json').strip())
<ipython-input-51-a66ee8866337>:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  new_df = pd.read_json(f.strip('`').lstrip('json').strip())
<ipython-input-51-a66ee8866337>:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  new_df = pd.read_json(f.strip('`').lstrip('json').strip())


In [52]:
df

,Name,Age,School,State,Project Title
0,Liam Aranda-Michel,18,Lake Oswego High School,OR,HemoLens: An Open-Source Pressure Myograph and...
1,Dev Arora,17,Plano West Senior High School,TX,A Novel Reinforcement-Learning Approach to Clo...
2,Asa Ashley,17,The Carol Martin Gatton Academy of Mathematics...,KY,Derived Magnetite on Nickel Foam as an Electro...
3,Chloe C. Au,17,Paul Duke STEM High School,GA,The Role of Myc in Tunicate Central Nervous Sy...
4,Naima Aubry-Romero,17,Richard Montgomery High School,MD,"Modeling, Analysis and Prediction of COVID-19 ..."
5,Fatimah Badoolah,17,Garden City Park School,NY,The Efficacy of Different Dosages of M. charan...
6,Sanchali Banerjee,18,Thomas Jefferson High School for Science and T...,VA,LUSO-GenAI: A Novel Framework Using Conditiona...
7,Laasya Acharya,17,William Mason High School,OH,Implementing a Novel Multimodal Neural Network...
8,Esat Acikgoz,17,Bergen County Academies,NJ,Cyanobacteria: Limiting the Bad and Exploiting...
9,Johnathan Ahdout,17,John L. Miller Great Neck North High School,NY,Contextualizing Data-Sparse QSPR Property Pred...
